# Branch-and-cut for the 0–1 Knapsack Problem

## Task 1: Primal Heuristic

In [ ]:
#testing variables:

p=[5,4,3,5,4,5]
w=[2,2,2,4,4,6,]

items = [
    {"w": 2, "p": 5},
    {"w": 2, "p": 4},
    {"w": 2, "p": 3},
    {"w": 4, "p": 5}
]

c = 10
a=c
a='hello'

In [16]:
import random

def generate_random_items(num_items):
    items = []
    for _ in range(num_items):
        # Generate random weights and profits within a similar range as the example
        weight = random.randint(1, 10)  # Random weight between 1 and 10
        profit = random.randint(1, 10)  # Random profit between 1 and 10
        items.append({"w": weight, "p": profit})
    return items

# Generate a list of 1000 items
random_items = generate_random_items(1000)

# Display the first 10 items to check
for item in random_items[:10]:
    print(item)


{'w': 5, 'p': 10}
{'w': 8, 'p': 10}
{'w': 7, 'p': 8}
{'w': 5, 'p': 4}
{'w': 8, 'p': 8}
{'w': 2, 'p': 4}
{'w': 7, 'p': 6}
{'w': 1, 'p': 10}
{'w': 10, 'p': 3}
{'w': 6, 'p': 10}


In [18]:
def knapsack_greedy_heuristic(items, c):
    #no need to sort
    items=sorted(items, key = lambda x: x['p']/x['w'], reverse = True)
    total_value=0
    total_weight=0
    packed_items = []
    for item in items:
        if total_weight + item['w']<=c:
            packed_items.append(item)
            total_weight += item['w']
            total_value += item ['p']
    return total_value, packed_items, total_weight

print(knapsack_greedy_heuristic(random_items, c))


(99, [{'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 10}, {'w': 1, 'p': 9}], 10)


Lets create our shortest path code. test

# OUR CODE 

In [6]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB

In [2]:
Vertex = int #(0,1,2,3,4,...)
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    x: List[float]
    y: List[float]
    cost: Dict[Arc, float]

    def __init__(self, x: List[float], y: List[float]):
        assert len(x) == len(y), "X and Y coordinate lists must have the same length"

        self.n = len(x)
        self.x = x
        self.y = y
        self.cost = {
            (i, j): ((self.x[i] - self.x[j])**2 + (self.y[i] - self.y[j])**2)**0.5
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }

    def vertices(self) -> Iterable[Vertex]:
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod #create a TSP instance to test our code
    def random(n: int) -> TSPIntance:
        x = [uniform(0, 10) for _ in range(n)]
        y = [uniform(0, 10) for _ in range(n)]
        return TSPIntance(x=x, y=y)

In [5]:
class TSPSolution:
    tour: Tour
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [ ]:
class BranchAndCutIntegerSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x')
        self.__build_model()

    def __build_model(self) -> None:
        
        self.m.addConstrs(self.x.sum(i, '*') == 1 for i in self.tsp.vertices())#sums ij: we add a constraint for all i's #constraint: all only one incoming trip to a customer
        self.m.addConstrs(self.x.sum('*', i) == 1 for i in self.tsp.vertices())#sums ji: we add a constraint #only one outgoing trip to customer

    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1) #lazy sonstraints are going to be used
        self.m.optimize(lambda _, where: self.__separate(where=where)) #optimize: solve this gruobi! we sepcify 2 parameters to check finding the subgroups 
        #lamda has two arguments: The model and the int number: _ (=ignoring the Model as we have 
        # it safed anyways in the class BranchAndCutIntegerSolver "model") and 

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __separate(self, where: int) -> None: #= __find_subtours
        if where != GRB.Callback.MIPSOL: #See documentations of callback online, SO: when we don't find a feasable int solution we return
            return
        
        remaining = set(self.tsp.vertices()) #the nodes we still have to explore, at first we have all of them

        while len(remaining) > 0:
            # Get the first vertex of the set
            start = remaining.pop()
            #start = next(iter(remaining))
            subtour = self.__tour_starting_at(start)

            if len(subtour) == self.tsp.n:
                # Feasible tour visiting all vertices
                return
            self.__add_sec_for(subtour)

            remaining -= set(subtour) #convert subtour into set because we cannot do set-= list

    def __tour_starting_at(self, i: Vertex) -> Tour:#e.g.: subset (2,3,7), i=2, tour =[2], current = 3 -next iteration: tour = [2,3], current = 7, -next iteration: tour = [2,3,7], current = 2 -
        #2 is not different form 2 so the last one is the subtour
        tour = [i]
        current = self.__next_vertex(i=i)

        while current != i:
            tour.append(current)
            current = self.__next_vertex(current)

        return tour

    def __next_vertex(self, i: Vertex) -> Vertex: #check which vertex we are going to 
        for j in self.tsp.vertices():#for all vertices except itself, check which one we are going to:
            if j == i:
                continue

            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j]) #we cannot use x[i,j].X when using callbacki
            except GurobiError: #if we fail we use x[i,j].X
                # When optimisation is over
                x = self.x[i,j].X #2 is not different form 2 s

            if x > 0.5:
                return j
            
        raise RuntimeError(f"Ve2 is not different form 2 srtex {i} has no successor!")
    
#m.cbGetSolution(x[i,j]) during a callback
#x[i,j].X after optimization
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding a SEC/subtour for [" + ', '.join(map(str, subtour)) + "]")
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs() #using arcs function that contains all valid arcs
                if i in subtour and j not in subtour
            ) >= 1
        )

In [6]:
tsp = TSPIntance.random(n=20)
solver = BranchAndCutIntegerSolver(tsp=tsp)
solution = solver.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 838 rows, 380 columns and 2584 nonzeros
Model fingerprint: 0xc60c6edd
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 93.2298364
Presolve removed 798 rows and 0 columns
Presolve time: 0.00s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 2.682548e+01, 31 iterations, 0.00 seconds (0.00 work units)
Adding a SEC/subtour for [0, 13, 4]
Adding a SEC/subtour for [1, 11]
Adding a SEC/subtour for [2, 5]
Adding 

### ChatGPT Solution, Branch and Cut

Model with negative costs. We see that it does not work.

In [2]:
import gurobipy as gp
from gurobipy import GRB

# A small graph with negative cycles that will result in an unbounded solution (problematic)
class NegativeCycleGraph:
    def __init__(self):
        # Define vertices
        self.vertices = [0, 1, 2, 3]
        
        # Define arcs and their associated costs (create a negative cycle)
        self.cost = {
            (0, 1): 10,  # Positive arc
            (1, 2): -15, # Negative cost arc
            (2, 3): 10,  # Positive arc
            (3, 1): 5,   # Positive arc
            (0, 3): 5,   # Positive arc
            (1, 3): 7    # Positive arc
        }

    def arcs(self):
        return self.cost.keys()

# Solve using optimization with Gurobi (integer programming)
def solve_negative_cycle_graph():
    graph = NegativeCycleGraph()
    
    # Create Gurobi model
    model = gp.Model()

    # Create binary variables for each arc (whether the arc is part of the tour)
    x = model.addVars(graph.arcs(), vtype=GRB.BINARY, name="x")
    
    # Objective function: Minimize the total cost of the selected arcs
    model.setObjective(gp.quicksum(graph.cost[arc] * x[arc] for arc in graph.arcs()), GRB.MINIMIZE)
    
    # Constraints: Each vertex should have exactly one incoming and outgoing arc (simple TSP formulation)
    for vertex in graph.vertices:
        model.addConstr(gp.quicksum(x[i, vertex] for i in graph.vertices if (i, vertex) in graph.arcs()) == 1)  # Incoming
        model.addConstr(gp.quicksum(x[vertex, j] for j in graph.vertices if (vertex, j) in graph.arcs()) == 1)  # Outgoing
    
    # Optimize the model
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        print(f"Optimal solution found with cost: {model.objVal}")
        for arc in graph.arcs():
            if x[arc].x > 0.5:  # If the arc is part of the solution
                print(f"Arc {arc} is part of the solution with cost {graph.cost[arc]}")
    else:
        print("No optimal solution found.")

# Call the function to see the result
solve_negative_cycle_graph()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 8 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xc19f0b78
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.


Model with positive costs. We see that it works.

In [22]:
from gurobipy import Model, GRB, quicksum
from typing import List, Tuple, Dict, Iterable

Vertex = int  # Nodes (0, 1, 2, 3, ...)
Arc = Tuple[Vertex, Vertex]

class SPPInstance:
    n: int
    cost: Dict[Arc, float]

    def __init__(self, n: int, arcs: List[Arc], costs: Dict[Arc, float]):
        self.n = n
        self.cost = costs
        self.arcs = arcs

    def vertices(self) -> Iterable[Vertex]:
        return range(self.n)


class BranchAndCutSPPSolver:
    spp: SPPInstance
    m: Model
    x: dict

    def __init__(self, spp: SPPInstance, source: Vertex, sink: Vertex):
        self.spp = spp
        self.source = source
        self.sink = sink
        self.m = Model()
        self.x = self.m.addVars(self.spp.arcs, vtype=GRB.BINARY, obj=self.spp.cost, name='x')
        self.__build_model()

    def __build_model(self) -> None:
        vertices = self.spp.vertices()

        # Flow constraints
        self.m.addConstr(quicksum(self.x[self.source, j] for j in vertices if (self.source, j) in self.spp.arcs) == 1, name="SourceFlow")
        self.m.addConstr(quicksum(self.x[i, self.sink] for i in vertices if (i, self.sink) in self.spp.arcs) == 1, name="SinkFlow")
        
        # Intermediate node flow conservation
        self.m.addConstrs(
            quicksum(self.x[i, j] for j in vertices if (i, j) in self.spp.arcs) ==
            quicksum(self.x[j, i] for j in vertices if (j, i) in self.spp.arcs)
            for i in vertices if i != self.source and i != self.sink
        )

    def solve(self):
        self.m.setParam(GRB.Param.LazyConstraints, 1)
        self.m.optimize(lambda _, where: self.__separate(where=where))

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve SPP model to optimality")

        solution_arcs = [arc for arc in self.spp.arcs if self.x[arc].X > 0.5]
        total_cost = sum(self.spp.cost[arc] for arc in solution_arcs)
        return solution_arcs, total_cost

    def __separate(self, where: int) -> None:
        if where != GRB.Callback.MIPSOL:
            return

        sol = self.m.cbGetSolution(self.x)

        # Build selected arcs graph
        graph = {v: [] for v in self.spp.vertices()}
        for i, j in self.spp.arcs:
            if sol[i, j] > 0.5:
                graph[i].append(j)

        # Check if the sink is reachable from the source
        visited = set()
        stack = [self.source]

        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                stack.extend(graph[node])

        if self.sink not in visited:
            # Add a cut to eliminate unreachable solutions
            unreachable = set(self.spp.vertices()) - visited
            cut_arcs = [(i, j) for i in visited for j in unreachable if (i, j) in self.spp.arcs]
            self.m.cbLazy(quicksum(self.x[arc] for arc in cut_arcs) >= 1)


# Example Usage
vertices = list(range(10))
arcs = [
    (0, 1), (1, 2), (2, 3), (3, 4), (4, 9),
    (0, 5), (5, 6), (6, 7), (7, 8), (8, 9)
]
costs = {arc: i + 1 for i, arc in enumerate(arcs)}  # Arbitrary costs
spp_instance = SPPInstance(n=10, arcs=arcs, costs=costs)

source = 0
sink = 9

solver = BranchAndCutSPPSolver(spp=spp_instance, source=source, sink=sink)
solution_arcs, total_cost = solver.solve()

print("Solution Arcs:", solution_arcs)
print("Total Cost:", total_cost)


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 10 columns and 20 nonzeros
Model fingerprint: 0xb40f8da5
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 15.0000000
Presolve removed 10 rows and 9 columns
Presolve time: 0.00s
Presolved: 0 rows, 1 columns, 0 nonzeros
Variable types: 0 continuous, 1 integer (1 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 15 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500

### CHATgpt, branch and cut alternative

In [19]:
from gurobipy import Model, GRB, quicksum
from typing import List, Tuple, Dict

Vertex = int
Arc = Tuple[Vertex, Vertex]
CostDict = Dict[Arc, float]

def branch_and_cut(vertices: List[Vertex], arcs: List[Arc], costs: CostDict, source: Vertex, sink: Vertex):
    """
    Implements a branch-and-cut algorithm for the shortest path problem.
    
    Parameters:
    - vertices: List of vertices in the graph.
    - arcs: List of valid arcs (directed edges).
    - costs: Dictionary mapping arcs to their respective costs.
    - source: Source node.
    - sink: Sink node.
    
    Outputs:
    - Optimal path and its cost if a solution exists.
    """
    # Print costs of all arcs
    print("Costs of all arcs:")
    for arc, cost in costs.items():
        print(f"Arc {arc}: Cost = {cost}")
    print("\n")
    
    # Initialize the model
    model = Model("Branch_And_Cut")
    
    # Add binary variables for each arc
    x = model.addVars(arcs, vtype=GRB.BINARY, obj=costs, name="x")
    
    # Objective: Minimize total cost
    model.setObjective(quicksum(costs[arc] * x[arc] for arc in arcs), GRB.MINIMIZE)
    
    # Initial constraints: Source and sink flow
    model.addConstr(quicksum(x[source, j] for j in vertices if (source, j) in arcs) == 1, name="Source_Flow")
    model.addConstr(quicksum(x[i, sink] for i in vertices if (i, sink) in arcs) == 1, name="Sink_Flow")
    
    # Lazy constraints: Flow conservation and subtour elimination
    def callback(model, where):
        if where == GRB.Callback.MIPSOL:
            # Extract solution
            sol = model.cbGetSolution(x)
            
            # Build the graph of selected arcs
            selected_arcs = [arc for arc in arcs if sol[arc] > 0.5]
            graph = {v: [] for v in vertices}
            for i, j in selected_arcs:
                graph[i].append(j)
            
            # Perform a depth-first search to find all reachable nodes from the source
            visited = set()
            stack = [source]
            while stack:
                node = stack.pop()
                if node not in visited:
                    visited.add(node)
                    stack.extend(graph[node])
            
            # Check if the sink is reachable
            if sink not in visited:
                # Add a cut: Ensure flow conservation for unreachable nodes
                unreachable = set(vertices) - visited
                cut_arcs = [(i, j) for i in visited for j in unreachable if (i, j) in arcs]
                if cut_arcs:
                    model.cbLazy(quicksum(x[arc] for arc in cut_arcs) >= 1)

    # Set callback for lazy constraints
    model.Params.LazyConstraints = 1
    model.optimize(callback)

    # Process the solution
    if model.status == GRB.OPTIMAL:
        solution = [arc for arc in arcs if x[arc].X > 0.5]
        total_cost = sum(costs[arc] for arc in solution)
        print(f"Optimal Path: {solution}")
        print(f"Total Cost: {total_cost}")
        return solution, total_cost
    else:
        print("No optimal solution found.")
        return None, None


# Example Usage
vertices = list(range(10))  # Nodes 0 through 9
arcs = [
    (0, 1), (1, 2), (2, 3), (3, 4), (4, 9),
    (0, 5), (5, 6), (6, 7), (7, 8), (8, 9)
]
costs = {arc: i + 1 for i, arc in enumerate(arcs)}  # Assign arbitrary costs

source = 0  # Source node
sink = 9    # Sink node

branch_and_cut(vertices, arcs, costs, source, sink)


Costs of all arcs:
Arc (0, 1): Cost = 1
Arc (1, 2): Cost = 2
Arc (2, 3): Cost = 3
Arc (3, 4): Cost = 4
Arc (4, 9): Cost = 5
Arc (0, 5): Cost = 6
Arc (5, 6): Cost = 7
Arc (6, 7): Cost = 8
Arc (7, 8): Cost = 9
Arc (8, 9): Cost = 10


Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 24.04.1 LTS")

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 10 columns and 4 nonzeros
Model fingerprint: 0x9c656c73
Variable types: 0 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 0 rows, 8 columns, 0 nonzeros
Variable types: 0 continuous, 8 integer (8 binary)

Root relaxation: objective 8.

([(0, 1), (1, 2), (2, 3), (3, 4), (4, 9)], 15)